<a href="https://colab.research.google.com/github/nimiw0821/DS-GA_1011_NLP/blob/master/HW3/hw3_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---   
# HW3 - Transfer learning

#### Due October 30, 2019

In this assignment you will learn about transfer learning. This technique is perhaps one of the most important techniques for industry. When a problem you want to solve does not have enough data, we use a different (larger) dataset to learn representations which can help us solve our task using the smaller task.

The general steps to transfer learning are as follows:

1. Find a huge dataset with similar characteristics to the problem you are interested in.
2. Choose a model powerful enough to extract meaningful representations from the huge dataset.
3. Train this model on the huge dataset.
4. Use this model to train on the smaller dataset.


### This homework has the following sections:
1. Question 1: MNIST fine-tuning (Parts A, B, C, D).
2. Question 2: Pretrain on Wikitext2 (Part A, B, C, D)
3. Question 3: Finetune on MNLI (Part A, B, C, D)
4. Question 4: Finetune using pretrained BERT (Part A, B, C)

---   
## Question 1 (MNIST transfer learning)
To grasp the high-level approach to transfer learning, let's first do a simple example using computer vision. 

The torchvision library has pretrained models (resnets, vggnets, etc) on the Imagenet dataset. Imagenet is a dataset
with 1.3 million images covering over 1000 classes of objects. When you use one of these models, the weights of the model initialize
with the weights saved from training on imagenet.

In this task we will:
1. Choose a pretrained model.
2. Freeze the model so that the weights don't change.
3. Fine-tune on a few labels of MNIST.   

#### Choose a model
Here we pick any of the models from torchvision

In [0]:
import torch
import torchvision.models as models

class Identity(torch.nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
        
    def forward(self, x):
        return x

# init the pretrained feature extractor
pretrained_resnet18 = models.resnet18(pretrained=True)

# we don't want the built in last layer, we're going to modify it ourselves
pretrained_resnet18.fc = Identity()

#### Freeze the model
Here we freeze the weights of the model. Freezing means the gradients will not backpropagate
into these weights.

By doing this you can think about the model as a feature extractor. This feature extractor outputs
a **representation** of an input. This representation is a matrix that encodes information about the input.

In [0]:
def freeze_model(model):
    for param in model.parameters():
        param.requires_grad = False
        
def unfreeze_model(model):
    for param in model.parameters():
        param.requires_grad = True
        
freeze_model(pretrained_resnet18)

#### Init target dataset
Here we define the dataset we are actually interested in.

In [0]:
import os
from torchvision import transforms
from torchvision.datasets import  MNIST
from torch.utils.data import DataLoader, random_split
import torch.nn.functional as F

In [0]:
#  train/val  split
mnist_dataset = MNIST(os.getcwd(), train=True, download=True, transform = transforms.Compose([transforms.Grayscale(3),transforms.ToTensor()]))
mnist_train, mnist_val = random_split(mnist_dataset, [55000, 5000])

mnist_train = DataLoader(mnist_train, batch_size=32)
mnist_val = DataLoader(mnist_val, batch_size=32)

# test split
mnist_test = DataLoader(MNIST(os.getcwd(), train=False, download=True, transform = transforms.Compose([transforms.Grayscale(3),transforms.ToTensor()])), batch_size=32)

### Part A (init fine-tune model)
decide what model to use for fine-tuning

In [0]:
def init_fine_tune_model():
  # we will use the structure from resnet18 as our fine_tune_model
  feature_extractor = models.resnet18(pretrained=True)
  num_ftrs = feature_extractor.fc.in_features
  # freeze the model
  freeze_model(feature_extractor)
  fine_tune_model = feature_extractor
  # add the last layer 
  fine_tune_model.fc = nn.Linear(num_ftrs,10)
  fine_tune_model= fine_tune_model.to(current_device)
  return fine_tune_model

### Part B (Fine-tune (Frozen))

The actual problem we care about solving likely has a different number of classes or is a different task altogether. Fine-tuning is the process of using the extracted representations (features) to solve this downstream task  (the task you're interested in).

To illustrate this, we'll use our pretrained model (on Imagenet), to solve the MNIST classification task.

There are two types of finetuning. 

#### 1. Frozen feature_extractor
In the first type we pretrain with the FROZEN feature_extractor and NEVER unfreeze it during finetuning.


#### 2. Unfrozen feature_extractor
In the second, we finetune with a FROZEN feature_extractor for a few epochs, then unfreeze the feature extractor and finish training.


In this part we will use the first version

In [0]:
# defining what device to use
num_gpus = torch.cuda.device_count()
if num_gpus > 0:
    current_device = 'cuda'
else:
    current_device = 'cpu'

In [0]:
import torch.optim as optim
import torch.nn as nn

def FROZEN_fine_tune_mnist(feature_extractor, fine_tune_model, mnist_train, mnist_val,model_name,hyperparameters,save):

  model =  fine_tune_model 

  # criterion
  criterion = nn.CrossEntropyLoss()
  # only update the parameters that are not frozen 
  model_params = [p for p in model.parameters() if p.requires_grad]
  optimizer = optim.Adam(model_params, lr=hyperparameters['lr'], weight_decay=hyperparameters['weight_decay'])
  num_epochs = hyperparameters['num_epochs']
  PATH = model_name + '.pth'

  for epoch in range(num_epochs):
    avg_loss = 0
    train_log_cache = []

    # training phase
    model.train()
    for i,(inp,target) in enumerate(mnist_train):
      optimizer.zero_grad()
      inp = inp.to(current_device)
      target = target.to(current_device)
      logits = model(inp)
      # compute loss 
      loss = criterion(logits.view(-1, logits.size(-1)), target.view(-1))
      # back-propogation 
      loss.backward()
      # gradient clipping 
      nn.utils.clip_grad_norm_(model.parameters(), max_norm=5)
      optimizer.step()
      train_log_cache.append(loss.item())

      if i % 1000 == 0:
        avg_train_loss = sum(train_log_cache)/len(train_log_cache)
        print('Step {} avg train loss = {:.{prec}f}'.format(i, avg_train_loss, prec=4))
        train_log_cache = []

    # validation phase 
    valid_losses = []
    model.eval()
    with torch.no_grad():
      for i,(inp,target) in enumerate(mnist_val):
        #inp = inp.repeat(1,3,1,1)
        inp = inp.to(current_device)
        target = target.to(current_device)
        logits = model(inp)
        # compute loss 
        loss = criterion(logits.view(-1, logits.size(-1)), target.view(-1))
        valid_losses.append(loss.item())
    avg_val_loss = sum(valid_losses) / len(valid_losses)
    print('Validation loss after {} epoch = {:.{prec}f}'.format(epoch+1, avg_val_loss, prec=4))

  print('Finish training!')

  if save:
    torch.save(model,PATH)

### Part C (compute test accuracy)
Compute the test accuracy of fine-tuned model on MNIST

In [0]:
  def calculate_mnist_test_accuracy(feature_extractor, fine_tune_model, mnist_test):
    model = fine_tune_model
    model.eval()
    test_accuracy = []
    batch_num = 0
    with torch.no_grad():
        for i, (inp, target) in enumerate(mnist_test):
          batch_num += 1
          inp = inp.to(current_device)
          target = target.to(current_device)
          logits = model(inp)
          softmax_output = F.softmax(logits,dim = 1)
          predictions = torch.argmax(softmax_output, dim=1)
          for i in range(len(predictions)):
            if predictions[i] == target[i]:
              test_accuracy.append(1)
            else:
              test_accuracy.append(0)
    
    return(sum(test_accuracy) / len(test_accuracy))

### Grade!
Let's see how you did

In [0]:
def grade_mnist_frozen():

    # init a ft model
    fine_tune_model = init_fine_tune_model()
    
    # run the transfer learning routine
    FROZEN_fine_tune_mnist(feature_extractor = pretrained_resnet18,
                          fine_tune_model = fine_tune_model,
                          mnist_train =  mnist_train,
                          mnist_val = mnist_val,
                          model_name = 'frozen_resnet18',
                          hyperparameters = hyperparameters,
                          save = True)
    
    # calculate test accuracy
    PATH = 'frozen_resnet18.pth'
    fine_tune_model = torch.load(PATH, map_location=current_device)
    test_accuracy = calculate_mnist_test_accuracy(pretrained_resnet18, fine_tune_model, mnist_test)
    
    # the real threshold will be released by Oct 11 
    assert test_accuracy > 0.0, 'your accuracy is too low...'
    
    return test_accuracy
    
hyperparameters= {
    'lr': 0.001,
    'num_epochs': 15,
    'weight_decay': 0         
 }    
frozen_test_accuracy = grade_mnist_frozen()

Step 0 avg train loss = 2.6672
Step 1000 avg train loss = 1.0820
Validation loss after 1 epoch = 0.8098
Step 0 avg train loss = 1.1259
Step 1000 avg train loss = 0.8539
Validation loss after 2 epoch = 0.7881
Step 0 avg train loss = 1.1120
Step 1000 avg train loss = 0.8295
Validation loss after 3 epoch = 0.7830
Step 0 avg train loss = 1.1153
Step 1000 avg train loss = 0.8197
Validation loss after 4 epoch = 0.7821
Step 0 avg train loss = 1.1225
Step 1000 avg train loss = 0.8149
Validation loss after 5 epoch = 0.7827
Step 0 avg train loss = 1.1299
Step 1000 avg train loss = 0.8122
Validation loss after 6 epoch = 0.7836
Step 0 avg train loss = 1.1363
Step 1000 avg train loss = 0.8107
Validation loss after 7 epoch = 0.7847
Step 0 avg train loss = 1.1416
Step 1000 avg train loss = 0.8098
Validation loss after 8 epoch = 0.7856
Step 0 avg train loss = 1.1457
Step 1000 avg train loss = 0.8092
Validation loss after 9 epoch = 0.7865
Step 0 avg train loss = 1.1490
Step 1000 avg train loss = 0.8089

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type ResNet. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Conv2d. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type BatchNorm2d. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type ReLU. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist

In [0]:
print("The test accuracy for frozen model is {}".format(frozen_test_accuracy))

The test accuracy for frozen model is 0.7533


### Part D (Fine-tune Unfrozen)
Now we'll learn how to train using the "unfrozen" approach.

In this approach we'll:
1. keep the feature_extract frozen for a few epochs (10)
2. Unfreeze it.
3. Finish training

In [0]:
def UNFROZEN_fine_tune_mnist(feature_extractor, fine_tune_model, model_name, mnist_train, mnist_val, hyperparameters,save):
  model = fine_tune_model
  criterion = nn.CrossEntropyLoss()
  # only update the parameters that are not frozen 
  model_params = [p for p in model.parameters() if p.requires_grad]
  optimizer = optim.Adam(model_params, lr=hyperparameters['lr'], weight_decay=hyperparameters['weight_decay'])
  num_epochs = hyperparameters['num_epochs']
  PATH = model_name + '.pth'
  
  for epoch in range(num_epochs):
    if epoch >10:
      # unfreeze the model when more than 10 epochs
      unfreeze_model(model)
    avg_loss = 0
    train_log_cache = []

    # training phase
    model.train()
    for i,(inp,target) in enumerate(mnist_train):
      optimizer.zero_grad()
      #inp = inp.repeat(1,3,1,1)
      inp = inp.to(current_device)
      target = target.to(current_device)
      logits = model(inp)
      # compute loss 
      loss = criterion(logits.view(-1, logits.size(-1)), target.view(-1))
      # back-propogation 
      loss.backward()
      # gradient clipping 
      nn.utils.clip_grad_norm_(model.parameters(), max_norm=5)
      optimizer.step()
      train_log_cache.append(loss.item())

      if i % 1000 == 0:
        avg_train_loss = sum(train_log_cache)/len(train_log_cache)
        print('Step {} avg train loss = {:.{prec}f}'.format(i, avg_train_loss, prec=4))
        train_log_cache = []

    # validation phase 
    valid_losses = []
    model.eval()
    with torch.no_grad():
      for i,(inp,target) in enumerate(mnist_val):
        #inp = inp.repeat(1,3,1,1)
        inp = inp.to(current_device)
        target = target.to(current_device)
        logits = model(inp)
        # compute loss 
        loss = criterion(logits.view(-1, logits.size(-1)), target.view(-1))
        valid_losses.append(loss.item())
    avg_val_loss = sum(valid_losses) / len(valid_losses)
    print('Validation loss after {} epoch = {:.{prec}f}'.format(epoch+1, avg_val_loss, prec=4))

  print('Finish training!')

  if save:
    torch.save(model,PATH)

### Grade UNFROZEN
Let's see if there's a difference in accuracy!

In [0]:
def grade_mnist_unfrozen():
  # init a ft model
    fine_tune_model = init_fine_tune_model()
    
    # run the transfer learning routine
    UNFROZEN_fine_tune_mnist(feature_extractor = pretrained_resnet18,
                              fine_tune_model = fine_tune_model,
                              mnist_train =  mnist_train,
                              mnist_val = mnist_val,
                              model_name = 'unfrozen_resnet18',
                              hyperparameters = hyperparameters,
                              save = True)
    
    # calculate test accuracy
    PATH = 'unfrozen_resnet18.pth'
    fine_tune_model = torch.load(PATH, map_location=current_device)
    test_accuracy = calculate_mnist_test_accuracy(pretrained_resnet18, fine_tune_model, mnist_test)
    
    # the real threshold will be released by Oct 11 
    assert test_accuracy > 0.0, 'your accuracy is too low...'
    
    return test_accuracy

hyperparameters= {
    'lr': 0.001,
    'num_epochs': 15,
    'weight_decay': 0         
 }
    
unfrozen_test_accuracy = grade_mnist_unfrozen()

NameError: ignored

In [0]:
assert unfrozen_test_accuracy > frozen_test_accuracy, 'the unfrozen model should be better'

--- 
# Question 2 (train a model on Wikitext-2)

Here we'll apply what we just learned to NLP. In this section we'll make our own feature extractor and pretrain it on Wikitext-2.

The WikiText language modeling dataset is a collection of over 100 million tokens extracted from the set of verified Good and Featured articles on Wikipedia. The dataset is available under the Creative Commons Attribution-ShareAlike License.

#### Part A
In this section you need to generate the training, validation and test split. Feel free to use code from your previous lectures.

### part A load text from torch db

In [0]:
import torch
from torch.utils.data import Dataset
from tqdm import tqdm
import pickle
import torch
import torchvision.models as models
import os
from torchvision import transforms
from torchvision.datasets import  MNIST
from torch.utils.data import DataLoader, random_split
import torch.nn.functional as F
import os
import json
try:
  import jsonlines
except:
  !pip install jsonlines
import numpy as np
from collections import defaultdict
from torch import nn
import numpy
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable as V

In [0]:
class LMDataset(Dataset):
    def __init__(self, list_of_token_lists):
        self.input_tensors = []
        self.target_tensors = []

        for sample in list_of_token_lists:
            self.input_tensors.append(torch.tensor([sample[:-1]], dtype=torch.long))
            self.target_tensors.append(torch.tensor([sample[1:]], dtype=torch.long))

    def __len__(self):
        return len(self.input_tensors)

    def __getitem__(self, idx):
        return (self.input_tensors[idx], self.target_tensors[idx])


def tokenize_dataset(datasets, dictionary):
    tokenized_datasets = {}
    for split, dataset in datasets.items():
        _current_dictified = []
        for l in tqdm(dataset):
            l = ['<bos>'] + l + ['<eos>']
            encoded_l = dictionary.encode_token_seq(l)
            _current_dictified.append(encoded_l)
        tokenized_datasets[split] = _current_dictified
    return tokenized_datasets

def tokenize_mnli_dataset(datasets, dictionary):
    tokenized_datasets = {}
    for split, dataset in datasets.items():
        _current_dictified = []
        for s1, s2 in tqdm(dataset):
            s1 = ['<bos>'] + s1 + ['<eos>']
            s2 = ['<bos>'] + s2 + ['<eos>']
            encoded_s1 = dictionary.encode_token_seq(s1)            
            encoded_s2 = dictionary.encode_token_seq(s2)
            _current_dictified.append([encoded_s1, encoded_s2])
        tokenized_datasets[split] = _current_dictified
    return tokenized_datasets

def pad_list_of_tensors(list_of_tensors, pad_token):
    max_length = max([t.size(-1) for t in list_of_tensors])
    padded_list = []
    for t in list_of_tensors:
        padded_tensor = torch.cat(
            [t, torch.tensor([[pad_token] * (max_length - t.size(-1))], dtype=torch.long)], dim=-1)
        padded_list.append(padded_tensor)

    padded_tensor = torch.cat(padded_list, dim=0)
    return padded_tensor


def pad_collate_fn(pad_idx, batch):
    input_list = [s[0] for s in batch]
    target_list = [s[1] for s in batch]
    input_tensor = pad_list_of_tensors(input_list, pad_idx)
    target_tensor = pad_list_of_tensors(target_list, pad_idx)
    return input_tensor, target_tensor


In [0]:
def load_wikitext(data_dir):
    import subprocess
    filename = os.path.join(data_dir, 'wikitext2-sentencized.json')
    if not os.path.exists(filename):
        os.makedirs(data_dir, exist_ok=True)
        url = "https://nyu.box.com/shared/static/9kb7l7ci30hb6uahhbssjlq0kctr5ii4.json"
        args = ['wget', '-O', filename, url]
        subprocess.call(args)
    raw_datasets = json.load(open(filename, 'r'))
    for name in raw_datasets:
        raw_datasets[name] = [x.split() for x in raw_datasets[name]]

    if os.path.exists(os.path.join(data_dir, 'vocab.pkl')):
        vocab = pickle.load(open(os.path.join(data_dir, 'vocab.pkl'), 'rb'))
    else:
        vocab = Dictionary(raw_datasets, include_valid=False)
        pickle.dump(vocab, open(os.path.join(data_dir, 'vocab.pkl'), 'wb'))

    tokenized_datasets = tokenize_dataset(raw_datasets, vocab)
    datasets = {name: LMDataset(ds) for name, ds in tokenized_datasets.items()}
    print("Vocab size: %d" % (len(vocab)))
    return raw_datasets, datasets, vocab


class Dictionary(object):
    def __init__(self, datasets, include_valid=False):
        self.tokens = []
        self.ids = {}
        self.counts = {}
        
        # add special tokens
        self.add_token('<bos>')
        self.add_token('<eos>')
        self.add_token('<pad>')
        self.add_token('<unk>')
        
        for line in tqdm(datasets['train']):
            for w in line:
                self.add_token(w)
                    
        if include_valid is True:
            for line in tqdm(datasets['valid']):
                for w in line:
                    self.add_token(w)
                            
    def add_token(self, w):
        if w not in self.tokens:
            self.tokens.append(w)
            _w_id = len(self.tokens) - 1
            self.ids[w] = _w_id
            self.counts[w] = 1
        else:
            self.counts[w] += 1

    def get_id(self, w):
        return self.ids[w]
    
    def get_token(self, idx):
        return self.tokens[idx]
    
    def decode_idx_seq(self, l):
        return [self.tokens[i] for i in l]
    
    def encode_token_seq(self, l):
        return [self.ids[i] if i in self.ids else self.ids['<unk>'] for i in l]
    
    def __len__(self):
        return len(self.tokens)

In [14]:
raw_datasets, datasets, vocab = load_wikitext(os.getcwd())
data_loaders = {name: DataLoader(datasets[name], batch_size=32, shuffle=True,
                                        collate_fn=lambda x: pad_collate_fn(vocab.get_id('<pad>'), x))
                        for name in datasets}

100%|██████████| 9708/9708 [00:00<00:00, 134345.26it/s]


Vocab size: 33178


In [0]:
from torchtext.datasets import WikiText2

def init_wikitext_dataset():
    """
    Fill in the details
    """
    
    wikitext_val_loaders = data_loaders['valid'] # YOUR CODE HERE
    wikitext_train_loaders = data_loaders['train'] # YOUR CODE HERE
    wikitext_test_loaders = data_loaders['test'] # YOUR CODE HERE
    
    return wikitext_train_loaders, wikitext_val_loaders, wikitext_test_loaders

wk2_train_loaders, wk2_val_loaders, wk2_test_loaders = init_wikitext_dataset()

In [16]:
print('WIKITEXT 2 dataset statistics')
print(f'training samples:{len(wk2_train_loaders)}')
print(f'val samples:{len(wk2_val_loaders)}')
print(f'test samples:{len(wk2_test_loaders)}')

WIKITEXT 2 dataset statistics
training samples:2447
val samples:265
test samples:304


#### Part B   
Here we design our own feature extractor. In MNIST that was a resnet because we were dealing with images. Now we need to pick a model that can model sequences better. Design an RNN-based model here.

In [0]:
embed_dim = 64
hidden_dim = 512
num_layers = 3
dropout = 0.1
options = {
    'vocab_size': len(vocab),
    'embed_dim': embed_dim,
    'padding_idx': vocab.get_id('<pad>'),
    'hidden_dim': hidden_dim,
    'num_layers': num_layers,
    'dropout': dropout,
    'batch_first': True,
    'num_classes': len(vocab)
}

hyperparams = {
    'optimizer': 'Adam',
    'lr': 0.001,
    'num_epochs': 5,
    'weight_decay': 0
}


feature_extractor_name = "feature_extractor"
PATH = feature_extractor_name + '.pth'

In [0]:
class LstmLM(torch.nn.Module):
    def __init__(self):
        '''
        params:
            @options: dictionary of model parameters
        '''
        super(LstmLM, self).__init__()
        self.hidden_dim = options['hidden_dim']
        self.vocab_size = options['vocab_size']
        self.padding_idx = options['padding_idx']
        self.num_layers = options['num_layers']
        self.batch_first = options['batch_first'] # boolean
        self.embed_dim = options['embed_dim']
        self.num_classes = options['num_classes'] # if during LM pretraining: = vocab_size
        self.p = options['dropout']
        
        self.lookup = nn.Embedding(self.vocab_size, self.embed_dim, self.padding_idx)
        self.lstm = nn.LSTM(self.embed_dim, self.hidden_dim, self.num_layers, batch_first=self.batch_first, dropout=self.p) # lstm takes word embeddings as inputs and outputs hidden states (dim=hidden_dinm)
        # up until now, we can us as feature extractor. The last linear layer maps from hidden states to word space needs to be defined differently for different task.
        self.projection = nn.Linear(self.hidden_dim, self.num_classes)
    
    def features(self, encoded_input_sequence):
        x = self.lookup(encoded_input_sequence)
        x, _ = self.lstm(x) # distributed representation
        return x

    def forward(self, encoded_input_sequence):
        '''
        Forwrad method process the input from token ids to logits
        params:
            @inp: input sentence
        '''
        x = self.features(encoded_input_sequence)
        x = self.projection(x)
        return x  #logits

In [0]:
def init_feature_extractor():
  '''
  @options: dictionary of model parameters
  '''
  #  YOUR CODE
  feature_extractor = LstmLM()
  feature_extractor = feature_extractor.to(current_device)
  return feature_extractor

#### Part C
Pretrain the feature extractor

In [0]:
def perplexity(loss):
  '''
  function that computes perplexity
  '''
  return 2**(loss/np.log(2))

In [0]:
def fit_feature_extractor(feature_extractor, wikitext_train, wikitext_val):
    '''
    function to train feature extractor
    params:
      @feature_extractor: initialzied feature extractor object
      @wikitext_train: train dataloader
      @wikitext_val: validation dataloader
    
    '''
    # FILL IN THE DETAILS
    model = feature_extractor

    print("Training {}:".format(feature_extractor_name))
    
    # criterion:
    criterion = nn.CrossEntropyLoss(ignore_index=vocab.get_id('<pad>'))
    if os.path.exists(PATH): # load pre-trained
        print("PATH exists!")
        model = torch.load(PATH)
        # model.load_state_dict(checkpoint['model_state_dict'])
        # optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    
    else:
      # optimizer:
      model_params = [p for p in model.parameters() if p.requires_grad]
      if hyperparams['optimizer'] == 'SGD':
          optimizer = optim.SGD(model_params, lr=hyperparams['lr'], momentum=hyperparams['momentum'])
      elif hyperparams['optimizer'] == 'Adam':
          optimizer = optim.Adam(model_params, lr=hyperparams['lr'], weight_decay=hyperparams['weight_decay'])

    plot_cache = []
    num_epochs = hyperparams['num_epochs']

    for epoch in range(num_epochs):
        avg_loss=0
        # do train
        model.train()
        train_log_cache = []
        for i, (inp, target) in enumerate(wikitext_train):
            optimizer.zero_grad()
            inp = inp.to(current_device)
            target = target.to(current_device)
            logits = model(inp)
            # compute loss
            loss = criterion(logits.view(-1, logits.size(-1)), target.view(-1))
            # back-propogation
            loss.backward()
            # gradient clipping
            nn.utils.clip_grad_norm_(model.parameters(), max_norm=5)
            optimizer.step()
            train_log_cache.append(loss.item()) # store training loss

            if i % 1000 == 0:
                avg_train_loss = sum(train_log_cache)/len(train_log_cache)
                avg_train_perplexity = perplexity(avg_train_loss)
                print('Step {} avg train loss = {:.{prec}f}'.format(i, avg_train_loss, prec=4))
                print('Step {} avg train perplexity = {:.{prec}f}'.format(i, avg_train_perplexity, prec=4))
                train_log_cache = []

        #do validation
        valid_losses = []
        model.eval()
        with torch.no_grad():
            for i, (inp, target) in enumerate(wikitext_val):
                # current_batch_size = len(inp)
                inp = inp.to(current_device)
                target = target.to(current_device)
                device = torch.device("cuda")
                logits = model(inp)
                # compute loss
                loss = criterion(logits.view(-1, logits.size(-1)), target.view(-1))
                valid_losses.append(loss.item()) # store validation loss
            avg_val_loss = sum(valid_losses) / len(valid_losses)
            avg_val_perplexity = perplexity(avg_val_loss)
            print('Validation loss after {} epoch = {:.{prec}f}'.format(epoch, avg_val_loss, prec=4))
            print('Validation perplexity after {} epoch = {:.{prec}f}'.format(epoch, avg_val_perplexity, prec=4))

        plot_cache.append((avg_train_loss, avg_val_loss, avg_train_perplexity, avg_val_perplexity))
    
    torch.save(model, PATH) # save the entire model object

    print('Finished training')
    return plot_cache, model

#### Part D
Calculate the test perplexity on wikitext2. Feel free to recycle code from previous assignments from this class. 

In [0]:
def calculate_wiki2_test_perplexity(feature_extractor, wikitext_test):
  model = torch.load('feature_extractor.pth')
  model.eval()
  criterion = nn.CrossEntropyLoss(ignore_index=vocab.get_id('<pad>'))
  test_losses = []
  test_perp = []
  with torch.no_grad():
    for i, (inp, target) in enumerate(wikitext_test):
      inp = inp.to(current_device)
      target = target.to(current_device)
      logits = model(inp)
      # compute loss
      test_loss = criterion(logits.view(-1, logits.size(-1)), target.view(-1))
      test_losses.append(test_loss.item())
      print(test_loss)
    avg_test_loss = sum(test_losses) / len(test_losses)
    avg_test_perplexity = perplexity(avg_test_loss)
    test_perp.append(avg_test_perplexity)
  test_ppl = sum(test_perp)/len(test_perp) # average perplexity
  print("average test perplexity: ", test_ppl)
  return test_ppl

#### Let's grade your results!
(don't touch this part)

In [29]:
def grade_wikitext2():
    # load data
    wikitext_train, wikitext_val, wikitext_test = init_wikitext_dataset()

    # load feature extractor
    feature_extractor = init_feature_extractor()

    # pretrain using the feature extractor
    fit_feature_extractor(feature_extractor, wikitext_train, wikitext_val)

    # check test accuracy
    test_ppl = calculate_wiki2_test_perplexity(feature_extractor, wikitext_test)

    # the real threshold will be released by Oct 11 
    assert test_ppl < 200, 'ummm... your perplexity is too high... ?should be less than 200'
    
grade_wikitext2()

Training feature_extractor:
Step 0 avg train loss = 10.4164
Step 0 avg train perplexity = 33403.9210
Step 1000 avg train loss = 6.8923
Step 1000 avg train perplexity = 984.6317
Step 2000 avg train loss = 6.2407
Step 2000 avg train perplexity = 513.2030
Validation loss after 0 epoch = 5.7646
Validation perplexity after 0 epoch = 318.8195
Step 0 avg train loss = 5.8389
Step 0 avg train perplexity = 343.4013
Step 1000 avg train loss = 5.7388
Step 1000 avg train perplexity = 310.6782
Step 2000 avg train loss = 5.5966
Step 2000 avg train perplexity = 269.5022
Validation loss after 1 epoch = 5.4527
Validation perplexity after 1 epoch = 233.3827
Step 0 avg train loss = 5.3085
Step 0 avg train perplexity = 202.0399
Step 1000 avg train loss = 5.2950
Step 1000 avg train perplexity = 199.3292
Step 2000 avg train loss = 5.2243
Step 2000 avg train perplexity = 185.7389
Validation loss after 2 epoch = 5.3345
Validation perplexity after 2 epoch = 207.3710
Step 0 avg train loss = 4.7688
Step 0 avg tra

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type LstmLM. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Embedding. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type LSTM. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Linear. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


NameError: ignored

---   
## Question 3 (fine-tune on MNLI)
In this question you will use your feature_extractor from question 2
to fine-tune on MNLI.

(From the website):
The Multi-Genre Natural Language Inference (MultiNLI) corpus is a crowd-sourced collection of 433k sentence pairs annotated with textual entailment information. The corpus is modeled on the SNLI corpus, but differs in that covers a range of genres of spoken and written text, and supports a distinctive cross-genre generalization evaluation. The corpus served as the basis for the shared task of the RepEval 2017 Workshop at EMNLP in Copenhagen.

MNLI has 3 genres (3 classes).
The goal of this question is to maximize the test accuracy in MNLI. 

### Part A
In this section you need to generate the training, validation and test split. Feel free to use code from your previous lectures.

In [0]:
y_label_map = {'contradiction':0,'neutral':1,'entailment':2}
import pandas as pd
import io
from collections import Counter

def get_string_tokenized_data(data):
    
    tokenized_data_x = [];
    y_labels = []
    all_tokens = [];
    
    for i,x in enumerate(data):
        label = x[2]
        if label == 'nan':
            continue
        
        label = y_label_map[label]
        y_labels.append(label)
        
        dp = [x[0].split(), x[1].split()]
        tokenized_data_x.append(dp)
        all_tokens += (dp[0] + dp[1])
        

    return all_tokens, tokenized_data_x, y_labels
        


# convert token to id in the dataset
def token2index_dataset(tokens_data, token2id):
    indices_data = []
    for tokens1, tokens2 in tokens_data:
        index_list1 = [token2id[token] if token in token2id else UNK_IDX for token in tokens1]
        index_list2 = [token2id[token] if token in token2id else UNK_IDX for token in tokens2]
        indices_data.append([index_list1, index_list2])
    return indices_data
  
# convert token to id in the dataset
def token2index_using_wikitext2_dict(tokens_data, vocab):
    indices_data = []
    for tokens1, tokens2 in tokens_data:
        index_list1 = vocab.encode_token_seq(tokens1)
        index_list2 = vocab.encode_token_seq(tokens2)
        indices_data.append([index_list1, index_list2])
    return indices_data


def build_vocab(all_tokens):
    # Returns:
    # id2token: list of tokens, where id2token[i] returns token that corresponds to token i
    # token2id: dictionary where keys represent tokens and corresponding values represent indices
    token_counter = Counter(all_tokens)
    vocab, count = zip(*token_counter.most_common(MAX_VOCAB_SIZE))
    id2token = list(vocab)
    token2id = dict(zip(vocab, range(2,2+len(vocab)))) 
    id2token = ['<pad>', '<unk>'] + id2token
    token2id['<pad>'] = PAD_IDX 
    token2id['<unk>'] = UNK_IDX
    return token2id, id2token

In [0]:
val_df = pd.read_csv('mnli_val.tsv', sep="\t")
print(val_df.head(2))

val_df  = np.array(val_df)
val_df = val_df.astype(str)
val_genre_list = val_df[:, 3]

                                           sentence1  ...      genre
0  'Not entirely , ' I snapped , harsher than int...  ...    fiction
1  cook and then the next time it would be my tur...  ...  telephone

[2 rows x 4 columns]


In [0]:
_, val_data_x, val_data_y = get_string_tokenized_data(val_df)
del val_df

In [0]:
train_df = pd.read_csv('mnli_train.tsv', sep="\t")

train_df  = np.array(train_df)
train_df = train_df.astype(str)
train_genre_list = train_df[:, 3]

In [0]:
_, train_data_x, train_data_y = get_string_tokenized_data(train_df)
del train_df

In [0]:
# token2id, id2token = build_vocab(all_fasttext_tokens)
# train_data_indices = token2index_using_wikitext2_dict(train_data_x, vocab)
# val_data_indices = token2index_using_wikitext2_dict(val_data_x, vocab)


mnli_raw_datasets = {'train': train_data_x, 'val': val_data_x}
mnli_tokenized_datasets = tokenize_mnli_dataset(mnli_raw_datasets, vocab)

train_data_indices = mnli_tokenized_datasets['train']
val_data_indices = mnli_tokenized_datasets['val']


# double checking
print('\n')
print ("Train dataset size is {}".format(len(train_data_indices)))
print ("Val dataset size is {}".format(len(val_data_indices)))

# del train_data_x
# del train_data_y
# del val_data_x
# del val_data_y
del mnli_tokenized_datasets

In [0]:
unique_genre = list(set(val_genre_list));
nb_classes = len(y_label_map)

In [0]:
MAX_SENTENCE_LENGTH = 200
class MNLIDataset(Dataset):
    """
    Class that represents a train/validation/test dataset that's readable for PyTorch
    Note that this class inherits torch.utils.data.Dataset
    """

    def __init__(self, data_x, target_list):
        """
        @param data_list: list of newsgroup tokens
        @param target_list: list of newsgroup targets

        """
        self.data_x = data_x;
        self.target_list = target_list
        
        assert(len(data_x) == len(target_list))

    def __len__(self):
        return len(self.target_list)

    def __getitem__(self, key):
        """
        Triggered when you call dataset[i]
        """
        prem_token_idx = self.data_x[key][0][:MAX_SENTENCE_LENGTH]
        hyp_token_idx = self.data_x[key][1][:MAX_SENTENCE_LENGTH]
        label = self.target_list[key]
        return [prem_token_idx, hyp_token_idx, label]


def encode_collate_func(batch):
    """
    Customized function for DataLoader that dynamically pads the batch so that all
    data have the same length
    """
    prem_data_list = []
    hyp_data_list = []
    label_list = []
    length_list = []
    # print("collate batch: ", batch[0][0])
    # batch[0][0] = batch[0][0][:MAX_SENTENCE_LENGTH]
    for datum in batch:
        label_list.append(datum[2])
    # padding
    for datum in batch:
        prem_padded_vec = np.pad(np.array(datum[0]),
                                 pad_width=((0, MAX_SENTENCE_LENGTH - len(datum[0]))),
                                 mode="constant", constant_values=0)
        hyp_padded_vec = np.pad(np.array(datum[1]),
                                pad_width=((0, MAX_SENTENCE_LENGTH - len(datum[1]))),
                                mode="constant", constant_values=0)
        prem_data_list.append(prem_padded_vec)
        hyp_data_list.append(hyp_padded_vec)
    return [torch.from_numpy((np.array(prem_data_list))), torch.from_numpy(np.array(hyp_data_list)),
            torch.LongTensor(label_list)]

In [0]:
BATCH_SIZE = 32
nb_train_samples = int(0.95 * len(train_data_indices))
nb_val_samples = len(train_data_indices) - nb_train_samples

# train/val split
train_val_dataset = MNLIDataset(train_data_indices, train_data_y)
train_dataset, val_dataset = random_split(train_val_dataset, [nb_train_samples, nb_val_samples])

# train loader
train_mnli_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=BATCH_SIZE,
                                           collate_fn=encode_collate_func,
                                           shuffle=True)

# val loader
val_mnli_loader = torch.utils.data.DataLoader(dataset=val_dataset,
                                           batch_size=BATCH_SIZE,
                                           collate_fn=encode_collate_func,
                                           shuffle=True)

# test loader
test_dataset = MNLIDataset(val_data_indices, val_data_y)
test_mnli_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                           batch_size=BATCH_SIZE,
                                           collate_fn=encode_collate_func,
                                           shuffle=True)

In [0]:
print('MNLI dataset statistics')
print(f'training samples:{len(train_mnli_loader)}')
print(f'val samples:{len(val_mnli_loader)}')
print(f'test samples:{len(test_mnli_loader)}')

In [0]:
from torchtext.datasets import MultiNLI

def init_mnli_dataset():
    """
    Fill in the details
    """
    mnli_val = None # TODO
    mnli_train = None # TODO
    mnli_test = None # TODO
    
    return mnli_train, mnli_val, mnli_test

### Part B
Here we again design a model for finetuning. Use the output of your feature-extractor as the input to this model. This should be a powerful classifier (up to you).

In [0]:
def init_finetune_model():
    # TODO FILL IN THE DETAILS
    feature_extractor = torch.load(PATH) # PATH needs to be self-defined
    # freeze the model
    freeze_model(feature_extractor)
    fine_tune_model = feature_extractor
    # add the last layer
    fine_tune_model.W = nn.Linear(feature_extractor.hidden_dim, mnli_num_classes)
    return fine_tune_model

### Part C
Use the feature_extractor and your fine_tune_model to fine_tune MNLI

In [0]:
# target classes: {0,1,2}
mnli_num_classes = 3

### Part D
Evaluate the test accuracy

In [0]:
def calculate_mnli_test_accuracy(feature_extractor, fine_tune_model, mnli_test):
    
    # YOUR CODE HERE...
    
    return test_ppl

### Let's grade your results

In [0]:
def grade_mnli():
    # load data
    mnli_train, mnli_val, mnli_test = init_mnli_dataset()

    # no need to load feature extractor because it is fine-tuned
    feature_extractor = feature_extractor

    # init the fine_tune model
    fine_tune_model = init_finetune_model()
    
    # finetune
    fine_tune_mnli(feature_extractor, fine_tune_model, mnli_train, mnli_val)

    # check test accuracy
    test_accuracy = calculate_mnli_test_accuracy(feature_extractor, wikitext_test)

    # the real threshold will be released by Oct 11 
    assert test_ppl > 0.00, 'ummm... your accuracy is too low...'
    
grade_mnli()

---  
## Question 4 (BERT)

A major direction in research came from a model called BERT, released last year.  

In this question you'll use BERT as your feature_extractor instead of the model you
designed yourself.

To get BERT, head on over to (https://github.com/huggingface/transformers) and load your BERT model here

In [0]:
mnli_num_classes = 3

In [21]:
!pip install transformers

In [22]:
from transformers.data.processors.glue import MnliProcessor
import torch
import pandas as pd
import os
import sys
import shutil
import argparse
import tempfile
import urllib.request
import zipfile
from transformers import glue_convert_examples_to_features as convert_examples_to_features
from transformers import BertTokenizer
from torch.utils.data import TensorDataset, RandomSampler, DataLoader


from transformers import (
    BertModel,
    BertTokenizer
)

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
bert = BertModel.from_pretrained('bert-base-cased', output_attentions=True)

In [0]:
TASKS = ["CoLA", "SST", "MRPC", "QQP", "STS", "MNLI", "SNLI", "QNLI", "RTE", "WNLI", "diagnostic"]
TASK2PATH = {
    "CoLA": "https://firebasestorage.googleapis.com/v0/b/mtl-sentence-representations.appspot.com/o/data%2FCoLA.zip?alt=media&token=46d5e637-3411-4188-bc44-5809b5bfb5f4",  # noqa
    "SST": "https://firebasestorage.googleapis.com/v0/b/mtl-sentence-representations.appspot.com/o/data%2FSST-2.zip?alt=media&token=aabc5f6b-e466-44a2-b9b4-cf6337f84ac8",  # noqa
    "MRPC": "https://firebasestorage.googleapis.com/v0/b/mtl-sentence-representations.appspot.com/o/data%2Fmrpc_dev_ids.tsv?alt=media&token=ec5c0836-31d5-48f4-b431-7480817f1adc",  # noqa
    "QQP": "https://firebasestorage.googleapis.com/v0/b/mtl-sentence-representations.appspot.com/o/data%2FQQP-clean.zip?alt=media&token=11a647cb-ecd3-49c9-9d31-79f8ca8fe277",  # noqa
    "STS": "https://firebasestorage.googleapis.com/v0/b/mtl-sentence-representations.appspot.com/o/data%2FSTS-B.zip?alt=media&token=bddb94a7-8706-4e0d-a694-1109e12273b5",  # noqa
    "MNLI": "https://firebasestorage.googleapis.com/v0/b/mtl-sentence-representations.appspot.com/o/data%2FMNLI.zip?alt=media&token=50329ea1-e339-40e2-809c-10c40afff3ce",  # noqa
    "SNLI": "https://firebasestorage.googleapis.com/v0/b/mtl-sentence-representations.appspot.com/o/data%2FSNLI.zip?alt=media&token=4afcfbb2-ff0c-4b2d-a09a-dbf07926f4df",  # noqa
    "QNLI": "https://firebasestorage.googleapis.com/v0/b/mtl-sentence-representations.appspot.com/o/data%2FQNLIv2.zip?alt=media&token=6fdcf570-0fc5-4631-8456-9505272d1601",  # noqa
    "RTE": "https://firebasestorage.googleapis.com/v0/b/mtl-sentence-representations.appspot.com/o/data%2FRTE.zip?alt=media&token=5efa7e85-a0bb-4f19-8ea2-9e1840f077fb",  # noqa
    "WNLI": "https://firebasestorage.googleapis.com/v0/b/mtl-sentence-representations.appspot.com/o/data%2FWNLI.zip?alt=media&token=068ad0a0-ded7-4bd7-99a5-5e00222e0faf",  # noqa
    "diagnostic": [
        "https://storage.googleapis.com/mtl-sentence-representations.appspot.com/tsvsWithoutLabels%2FAX.tsv?GoogleAccessId=firebase-adminsdk-0khhl@mtl-sentence-representations.iam.gserviceaccount.com&Expires=2498860800&Signature=DuQ2CSPt2Yfre0C%2BiISrVYrIFaZH1Lc7hBVZDD4ZyR7fZYOMNOUGpi8QxBmTNOrNPjR3z1cggo7WXFfrgECP6FBJSsURv8Ybrue8Ypt%2FTPxbuJ0Xc2FhDi%2BarnecCBFO77RSbfuz%2Bs95hRrYhTnByqu3U%2FYZPaj3tZt5QdfpH2IUROY8LiBXoXS46LE%2FgOQc%2FKN%2BA9SoscRDYsnxHfG0IjXGwHN%2Bf88q6hOmAxeNPx6moDulUF6XMUAaXCSFU%2BnRO2RDL9CapWxj%2BDl7syNyHhB7987hZ80B%2FwFkQ3MEs8auvt5XW1%2Bd4aCU7ytgM69r8JDCwibfhZxpaa4gd50QXQ%3D%3D",  # noqa
        "https://www.dropbox.com/s/ju7d95ifb072q9f/diagnostic-full.tsv?dl=1",
    ],
}

MRPC_TRAIN = "https://dl.fbaipublicfiles.com/senteval/senteval_data/msr_paraphrase_train.txt"
MRPC_TEST = "https://dl.fbaipublicfiles.com/senteval/senteval_data/msr_paraphrase_test.txt"


def download_and_extract(task, data_dir):
    print("Downloading and extracting %s..." % task)
    data_file = "%s.zip" % task
    urllib.request.urlretrieve(TASK2PATH[task], data_file)
    with zipfile.ZipFile(data_file) as zip_ref:
        zip_ref.extractall(data_dir)
    os.remove(data_file)
    print("\tCompleted!")


In [24]:
download_and_extract('MNLI', '.')

	Completed!


In [0]:
processor = MnliProcessor()

In [0]:
def generate_mnli_bert_dataloaders():
  # ----------------------
  # TRAIN/VAL DATALOADERS
  # ----------------------
  train = processor.get_train_examples('MNLI')
  features = convert_examples_to_features(train,
                                          tokenizer,
                                          label_list=['contradiction','neutral','entailment'],
                                          max_length=128,
                                          output_mode='classification',
                                          pad_on_left=False,
                                          pad_token=tokenizer.pad_token_id,
                                          pad_token_segment_id=0)
  train_dataset = TensorDataset(torch.tensor([f.input_ids for f in features], dtype=torch.long), 
                                torch.tensor([f.attention_mask for f in features], dtype=torch.long), 
                                torch.tensor([f.token_type_ids for f in features], dtype=torch.long), 
                                torch.tensor([f.label for f in features], dtype=torch.long))

  nb_train_samples = int(0.95 * len(train_dataset))
  nb_val_samples = len(train_dataset) - nb_train_samples

  bert_mnli_train_dataset, bert_mnli_val_dataset = random_split(train_dataset, [nb_train_samples, nb_val_samples])

  # train loader
  train_sampler = RandomSampler(bert_mnli_train_dataset)
  bert_mnli_train_dataloader = DataLoader(bert_mnli_train_dataset, sampler=train_sampler, batch_size=32)

  # val loader
  val_sampler = RandomSampler(bert_mnli_val_dataset)
  bert_mnli_val_dataloader = DataLoader(bert_mnli_val_dataset, sampler=val_sampler, batch_size=32)


  # ----------------------
  # TEST DATALOADERS
  # ----------------------
  dev = processor.get_dev_examples('MNLI')
  features = convert_examples_to_features(dev,
                                          tokenizer,
                                          label_list=['contradiction','neutral','entailment'],
                                          max_length=128,
                                          output_mode='classification',
                                          pad_on_left=False,
                                          pad_token=tokenizer.pad_token_id,
                                          pad_token_segment_id=0)

  bert_mnli_test_dataset = TensorDataset(torch.tensor([f.input_ids for f in features], dtype=torch.long), 
                                torch.tensor([f.attention_mask for f in features], dtype=torch.long), 
                                torch.tensor([f.token_type_ids for f in features], dtype=torch.long), 
                                torch.tensor([f.label for f in features], dtype=torch.long))

  # test dataset
  test_sampler = RandomSampler(bert_mnli_test_dataset)
  bert_mnli_test_dataloader = DataLoader(bert_mnli_test_dataset, sampler=test_sampler, batch_size=32)
  
  return bert_mnli_train_dataloader, bert_mnli_val_dataloader, bert_mnli_test_dataloader

### Part A (init BERT)
In this section you need to create an instance of BERT and return if from the function

In [0]:
class BERTSequenceClassifier(nn.Module):
    def __init__(self, bert, num_classes):
        super().__init__()
        self.bert = bert
        self.W = nn.Linear(bert.config.hidden_size, num_classes)
        self.num_classes = num_classes
        
    def forward(self, input_ids, attention_mask, token_type_ids):
        h, _, attn = self.bert(input_ids=input_ids, 
                               attention_mask=attention_mask, 
                               token_type_ids=token_type_ids)
        h_cls = h[:, 0]
        logits = self.W(h_cls)
        return logits, attn

In [0]:
from transformers import BertTokenizer, BertModel, BertForMaskedLM

def init_bert():
    
    BERT = BERTSequenceClassifier(bert, mnli_num_classes) # ... YOUR CODE HERE
    
    return BERT

In [30]:
feature_extractor = init_bert()
feature_extractor.bert.config.hidden_size

768

### Part B (fine-tune with BERT)

Use BERT as your feature extractor to finetune MNLI. Use a new finetune model (reset weights).

In [0]:
def init_finetune_model():
    # TODO FILL IN THE DETAILS
    feature_extractor = init_bert() # bert as feature extractor
    # freeze the model
    freeze_model(feature_extractor)
    fine_tune_model = feature_extractor
    # add the last layer for target task
    fine_tune_model.W = nn.Linear(feature_extractor.bert.config.hidden_size, mnli_num_classes)
    return fine_tune_model

In [0]:
hyperparams = {
    'optimizer': 'Adam',
    'lr': 2e-05,
    'num_epochs': 3,
    'weight_decay': 0
}

In [0]:
def fine_tune_mnli_BERT(BERT_feature_extractor, fine_tune_model, mnli_train, mnli_val):
    '''
    function to train feature extractor
    params:
      @BERT_feature_extractor: initialzied feature extractor object
      @fine_tune_model: initialized fine tune model for target task
      @mnli_train: train dataloader
      @mnli_val: validation dataloader
    
    '''
    # FILL IN THE DETAILS
    model = fine_tune_model
    model.to(current_device)
    PATH = 'bert_mnli.pth'
    print("Training BERT on MNLI......")
    
    # criterion:
    criterion = nn.CrossEntropyLoss(ignore_index=vocab.get_id('<pad>'))
    if os.path.exists(PATH): # load pre-trained
        print("PATH exists!")
        model = torch.load(PATH)
    
    else:
      # optimizer:
        model_params = [p for p in model.parameters() if p.requires_grad]
        if hyperparams['optimizer'] == 'SGD':
            optimizer = optim.SGD(model_params, lr=hyperparams['lr'], momentum=hyperparams['momentum'])
        elif hyperparams['optimizer'] == 'Adam':
            optimizer = optim.Adam(model_params, lr=hyperparams['lr'], weight_decay=hyperparams['weight_decay'])

    num_epochs = hyperparams['num_epochs']
    for epoch in range(num_epochs):
        # print("Epoch ", epoch)
        # if epoch >10:
        #   # unfreeze the model when more than 10 epochs
        unfreeze_model(model)
        # do train
        model.train()
        for i, batch in enumerate(mnli_train):
            train_correct = 0
            train_total = 0
            optimizer.zero_grad()
            batch = tuple(t.to(current_device) for t in batch)
            logits, _ = model(*batch[:-1])
            labels = batch[-1]
            # compute loss
            loss = criterion(logits.view(-1, logits.size(-1)), labels.view(-1))
            # store prediction accuracy
            softmax_output = F.softmax(logits, dim=1)
            predicted = softmax_output.max(1, keepdim=True)[1]
            train_total += labels.size(0)
            train_correct += predicted.eq(labels.view_as(predicted)).sum().item()
            train_acc = 100*train_correct/train_total
            # back-propogation
            loss.backward()
            # gradient clipping
            nn.utils.clip_grad_norm_(model.parameters(), max_norm=5)
            optimizer.step()

            if i % 1000 == 0:
              print('Average training acc = {} at step {}'.format(train_acc, i))

        #do validation
        model.eval()
        with torch.no_grad():
            for i, batch in enumerate(mnli_val):
                val_correct = 0
                val_total = 0
                optimizer.zero_grad()
                batch = tuple(t.to(current_device) for t in batch)
                logits, _ = model(*batch[:-1])
                labels = batch[-1]
                # compute loss
                loss = criterion(logits.view(-1, logits.size(-1)), labels.view(-1))
                valid_losses.append(loss.item()) # store validation loss
                # store prediction accuracy
                softmax_output = F.softmax(logits,dim = 1)
                predicted = softmax_output.max(1, keepdim=True)[1]
                val_total += labels.size(0)
                val_correct += predicted.eq(labels.view_as(predicted)).sum().item()
                valid_acc = 100 * val_correct / val_total
        print('Average validation acc after epcoh {} = {}'.format(epoch, valid_acc))
    
    torch.save(model, PATH) # save the entire model object

    print('Finished training')
    return model

### Part C
Evaluate how well we did

In [0]:
def calculate_mnli_test_accuracy_BERT(feature_extractor, fine_tune_model, mnli_test):
  # YOUR CODE HERE...
  model = fine_tune_model
  model.eval()
  test_accuracy = []
  with torch.no_grad():
    for i, batch in enumerate(mnli_test):
      optimizer.zero_grad()
      batch = tuple(t.to(current_device) for t in batch)
      logits, _ = model(*batch[:-1])
      labels = batch[-1]
      softmax_output = F.softmax(logits,dim = 1)
      predictions = torch.argmax(softmax_output, dim=1)
      for i in range(len(predictions)):
        if predictions[i] == labels[i]:
          test_accuracy.append(1)
        else:
          test_accuracy.append(0)

  return(sum(test_accuracy) / len(test_accuracy))

### Let's grade your BERT results!

In [37]:
BERT_feature_extractor = init_bert()

# load data
mnli_train, mnli_val, mnli_test = generate_mnli_bert_dataloaders()
print("(1). finished loading data...")

(1). finished loading data...


In [38]:
# init the fine_tune model
fine_tune_model = init_finetune_model()
print("(2). finished initiating fine tune model")

(2). finished initiating fine tune model


In [0]:
# finetune
fine_tune_mnli_BERT(BERT_feature_extractor, fine_tune_model, mnli_train, mnli_val)
print("(3). finished fine tuning")

Training BERT on MNLI......
Average training acc = 25.0 at step 0
Average training acc = 28.125 at step 1000
Average training acc = 37.5 at step 2000
Average training acc = 40.625 at step 3000


In [0]:
# check test accuracy
test_accuracy = calculate_mnli_test_accuracy(feature_extractor, mnli_test)
print("(4). finished test accuracy calculation")

# the real threshold will be released by Oct 11 
assert test_accuracy > 80, 'ummm... your accuracy is too low...'

In [80]:
def grade_mnli_BERT():
    BERT_feature_extractor = init_bert()
    
    # load data
    mnli_train, mnli_val, mnli_test = generate_mnli_bert_dataloaders()
    print("(1). finished loading data...")

    # init the fine_tune model
    fine_tune_model = init_finetune_model()
    print("(2). finished initiating fine tune model")
    
    # finetune
    fine_tune_mnli_BERT(BERT_feature_extractor, fine_tune_model, mnli_train, mnli_val)
    print("(3). finished fine tuning")

    # check test accuracy
    test_accuracy = calculate_mnli_test_accuracy(feature_extractor, mnli_test)
    print("(4). finished test accuracy calculation")
    
    # the real threshold will be released by Oct 11 
    assert test_accuracy > 0.8, 'ummm... your accuracy is too low...'
    
grade_mnli_BERT()

(1). finished loading data...
(2). finished initiating fine tune model
Training BERT on MNLI......


ValueError: ignored